In [ ]:
# Copyright © 2023 "Bronte" Sihan Li

# Introduction

In this project, we focus on applying clustering to different data sets and exploring clustering mechanisms, distance metrics, and before and after PCA analysis.
We work with the following datasets:
* [Liver Disorders. (1990). UCI Machine Learning Repository.](https://archive-beta.ics.uci.edu/dataset/60/liver+disorders)
* [UCI Activity Recognition Data Set](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)

## Dependencies

First, we import all of the dependencies used in our analysis, including the helper modules we have built for this project:
* `nearest_neighbor`: our own implementation of K nearest neighbor clustering 
* `cluster_quality` : calculates clustering quality with metrics including Rissannon Minimum Description Length, Krzanowski and Lai, or Ray and Turi
* `distance`: calculates distance between a vector and a set of vectors

In [1]:
import sys, os
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
os.chdir('..')
from scipy import stats
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  homogeneity_score, completeness_score, adjusted_rand_score
from project1.utility.pca import PCA
from project1.utility import train_test_split_csv
from project2.utility.nearest_neighbor import KNearestNeighbor as knn
from project2.utility.cluster_quality import cluster_quality, get_representation_error

# Apply Nearest Neighbor classification To Our Data Set


## Convert Continuous Target to Discrete Values
First we load our dataset.
In order to perform clustering and evaluate the results for the liver disorder dataset, we need to first convert to continuous target variable to discrete values.
Here is what our target distribution looks like:

In [2]:
# Load the liver disorders dataset
liver_data = pd.read_csv('project2/data/liver_disorder.csv', header=None)
# Add headers
FEATURES = ['mean corpuscular volume', 'alkaline phosphotase', 'alanine aminotransferase', 
            'aspartate aminotransferase', 'gamma-glutamyl transpeptidase', ]
TARGET = 'number of half-pint equivalents of alcoholic beverages drunk per day'

liver_data.columns = FEATURES + [TARGET] + ['train_test']

# Plot a histogram of the target values
fig = px.histogram(liver_data, x=TARGET, marginal='box',)
fig.show()



We do this by calculating the optimal histogram bin edges and applying the bin cuts to our target column. Here is the result:

In [3]:
# Convert target column to discrete values

# First, calculate the optimal integer bin edges, we arbitrarily choose 5 bins
bins = np.histogram_bin_edges(liver_data[TARGET], bins=5)
# Cut the target column into bins
bin_labels = [i for i in range(len(bins) - 1)]
liver_data['discrete_target'] = pd.cut(liver_data[TARGET], bins=bins, labels=bin_labels, include_lowest=True)

In [4]:
fig = px.histogram(liver_data, x='discrete_target', marginal='box')
fig.show()

## Our Distance Metric

In `distance.py`, there are a few implementations of the most common distance metrics in machine learning for numeric values:

* Euclidean: 
  
The formula for Euclidean distance is:
![Formula for Euclidean Distance between two vectors in n-dimensional space](images/Euclidean%20Distance.png)

When calculating the Euclidean distances for every vector between two matrices, we can simplify the equation for the distance matrix to:
![General formula for EDM](https://miro.medium.com/max/720/1*6eqyEbgXdY4wlynIpXJD5w.webp)

As all of our features for the liver disorder data set are numeric, we choose the Euclidean formula as our distance metric.

* Cosine

Give two vectors A and B, We can visualize the angle between them as theta:

![Angle between A and B](https://miro.medium.com/max/720/1*AqWG1VHnLf8P7_H4KDBwVQ.webp)

As the name suggests, the cosine distance between two vectors are calculated as follows:
![Cosine of angle between vector A and B](https://miro.medium.com/max/720/1*LfW66-WsYkFqWc4XYJbEJg.webp)


* Minkowski

With Minkowski distance we can define the order, p, and calculate the distance as:
![Minkowski distance between X and Y vector in n-dimensional space](https://wikimedia.org/api/rest_v1/media/math/render/svg/91ba4de01dd46e90d647cfa5d48343214bee9e43)

When p=1, this corresponds to the Manhattan distance and when p=2, this is the same as Euclidean distance.

## Implement nearest neighbor classification

Now, we will implement the nearest neighbor classifier from our helper utility module `nearest_neighbor`.
First, we set the number of neighbors to use (k) as 1.

In [5]:
# Initialize the KNN classifier
knn_liver = knn(k=1)
# Split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(liver_data.drop(columns=[TARGET, 'train_test', 'discrete_target']),
                                   liver_data['discrete_target'], test_size=0.25, random_state=0)

y_train = y_train.to_frame()
y_test = y_test.to_frame()

# Fit the model and predict the labels
y_pred = knn_liver.fit_predict(X_train, y_train, X_test, y_test)
accuracy = knn_liver.get_accuracy(y_pred, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.5977011494252874


To take a further look at how our classifier did, we can compute the confusion matrix and plot it as a heatmap:

In [6]:
knn_liver.confusion_matrix(y_pred, y_test)

Actual,0,1,2,4
Predicted,,,,
0,50,9,0,0
1,15,2,3,1
2,4,0,0,0
3,0,2,0,0
4,0,1,0,0


In [7]:
# Plot the confusion matrix
fig = px.imshow(knn_liver.confusion_matrix(y_pred, y_test), labels=dict(x='Actual', y='Predicted', color='Count',),
                aspect='auto', text_auto=True)
fig.show()

## Experiment With Our Classifier


In [8]:
# Use a higher k value for the KNN classifier
knn_liver2 = knn(k=5)
y_pred2 = knn_liver2.fit_predict(X_train, y_train, X_test, y_test)
accuracy2 = knn_liver2.get_accuracy(y_pred2, y_test)
print(f'Accuracy: {accuracy2}')

Accuracy: 0.7241379310344828


In [9]:
# Plot the new confusion matrix
fig = px.imshow(knn_liver2.confusion_matrix(y_pred2, y_test), labels=dict(x='Actual', y='Predicted', color='Count',),
                aspect='auto', text_auto=True)
fig.show()

# Experiment with clustering and PCA on two structured data sets
We explore finding the optimal number of clusters in a data set, how PCA project can affect clustering, and how different clustering algorithms perform on the same data.

## Plot Example Data
We can visualize the data sets and look for natural clusters using a scatter plot:

In [10]:
# Load the data
data_a = pd.read_csv('project2/data/clusterDataA-1.csv')
data_b = pd.read_csv('project2/data/clusterDataB-1.csv')

# plot the data for data_a
fig1 = px.scatter(data_a, x='X1', y='X2')
fig1.show()


In [11]:
# plot the data for data_b
fig2 = px.scatter(data_b, x='X1', y='X2')
fig2.show()

## Clustering The Example Data

In [12]:
# cluster the data for dataset A
kmeans_a = KMeans(n_clusters=6, random_state=0).fit(data_a)
# plot the clustered data for dataset A
fig_a = px.scatter(data_a, x='X1', y='X2', color=kmeans_a.labels_, color_continuous_scale='peach')
fig_a.show()

In [13]:
# cluster the data for dataset B
kmeans_b = KMeans(n_clusters=6, random_state=0).fit(data_b)
# plot the clustered data for dataset B
fig_b = px.scatter(data_b, x='X1', y='X2', color=kmeans_b.labels_, color_continuous_scale='PuBu')
fig_b.show()

Now we apply a different typing of cluster algorithm, complete linkage, a method of hierarchical cluster analysis,  to see if the results might be different.

In [14]:
agg_a = AgglomerativeClustering(n_clusters=6, linkage='complete')
assignments_a = agg_a.fit_predict(data_a)
agg_b = AgglomerativeClustering(n_clusters=6, linkage='complete')
assignments_b = agg_b.fit_predict(data_b)

In [15]:
fig_agg_a = px.scatter(data_a, x='X1', y='X2', color=assignments_a, color_continuous_scale='peach')
fig_agg_a.show()

In [16]:
fig_agg_b = px.scatter(data_b, x='X1', y='X2', color=assignments_b, color_continuous_scale='PuBu')
fig_agg_b.show()

As we can see, the grouping of data points are largely the same, however, the output cluster labels are very different between the two clustering methods. This corresponds with the difference in how the algorithms find the clusters: hierarchical clustering starts with a root cluster and branches out; whereas k-means clustering initializes cluster means and adjusts clustering based on the updated means.

## Compare Different Number of Clusters

How do we know how many clusters to choose for our data? One way to evaluate our clustering is by looking at the representation error with a range of k.
In this section, we compare different values of k and compute the representation error.

In [17]:
sse_a = np.zeros(9)
sse_b = np.zeros(9)
cluster_qualities_a = np.zeros(9)
cluster_qualities_b = np.zeros(9)

for i in range(2, 11):
    kmeans_a = KMeans(n_clusters=i, random_state=0).fit(data_a[['X1', 'X2']])
    sse_a[i-2] = get_representation_error(data_a[['X1', 'X2']].to_numpy(), kmeans_a)
    cq_a = cluster_quality(data_a[['X1', 'X2']].to_numpy(), kmeans_a, metric='ray-turi')
    cluster_qualities_a[i-2] = cq_a
    kmeans_b = KMeans(n_clusters=i, random_state=0).fit(data_b[['X1', 'X2']])
    sse_b[i-2] = get_representation_error(data_b[['X1', 'X2']].to_numpy(), kmeans_b)
    cq_b = cluster_quality(data_b[['X1', 'X2']].to_numpy(), kmeans_b, metric='ray-turi')
    cluster_qualities_b[i-2] = cq_b

Here is the resulting SSE (Sum Squared Error) from a k value ranging from 2 to 1 for data set A and B:

In [18]:
# plot SSE for dataset A and B
fig_sse_a = px.line(x=range(2, 11), y=sse_a, title='SSE for dataset A', labels={'x': 'Number of clusters', 'y': 'SSE'})
fig_sse_a.show()
fig_sse_b = px.line(x=range(2, 11), y=sse_b, title='SSE for dataset B', labels={'x': 'Number of clusters', 'y': 'SSE'})
fig_sse_b.show()

Now we can visualize the cluster quality metric (Ray-Turi) as well:

In [19]:
fig_cq_a = px.line(x=range(2, 11), y=cluster_qualities_a, title='Cluster Quality for dataset A',
                     labels={'x': 'Number of clusters', 'y': 'Cluster Quality'})
fig_cq_a.show()

fig_cq_b = px.line(x=range(2, 11), y=cluster_qualities_b, title='Cluster Quality for dataset B',
                        labels={'x': 'Number of clusters', 'y': 'Cluster Quality'})
fig_cq_b.show()

## Apply PCA to Example Data

To further analyze, we apply PCA to our example data and visualize the first eigenvector for each data set:

In [20]:
# perform PCA on dataset A
data_a = data_a[['X1', 'X2']]
pca_a = PCA(data=data_a)
projected_a = pca_a.do_pca()
first_eigen_vector_a = pca_a.components[0]
print(first_eigen_vector_a)

# plot the data for dataset A and the first eigenvector
fig_a = go.Figure()
fig_a.add_trace(go.Scatter(x=data_a['X1'], y=data_a['X2'], mode='markers', name='Data Points'))
fig_a.add_trace(go.Scatter(x=[0, first_eigen_vector_a[0]], y=[0, first_eigen_vector_a[1]],
                           mode='lines', line=dict(color='red', width=2), name='First Eigenvector'))
fig_a.update(layout=dict(title='PCA for dataset A', xaxis_title='X1', yaxis_title='X2'))
fig_a.show()


Mean: [0.23       0.08988889]
Standard Deviation: [3.44805017 2.72489775]
Eigenvalues: [1.09553917 0.91563402]
Eigenvectors: 
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
[0.70710678 0.70710678]


In [21]:
# perform PCA on dataset B
data_b = data_b[['X1', 'X2']]
pca_b = PCA(data=data_b)
projected_b = pca_b.do_pca()
first_eigen_vector_b = pca_b.components[0]
print(first_eigen_vector_b)

# plot the data for dataset B and the first eigenvector
fig_b = go.Figure()
fig_b.add_trace(go.Scatter(x=data_b['X1'], y=data_b['X2'], mode='markers', name='Data Points'))
fig_b.add_trace(go.Scatter(x=[0, first_eigen_vector_b[0]], y=[0, first_eigen_vector_b[1]],
                            mode='lines', line=dict(color='red', width=2), name='First Eigenvector'))
fig_b.update(layout=dict(title='PCA for dataset B', xaxis_title='X1', yaxis_title='X2'))
fig_b.show()

Mean: [-0.75455556  0.50227778]
Standard Deviation: [2.32178531 3.50371816]
Eigenvalues: [1.33428193 0.67689125]
Eigenvectors: 
[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
[0.70710678 0.70710678]


The direction of the eigenvector makes sense as it points out the direction of the most variance in our data set, which is more apparent in the graph for data set B.

## Re-clustering With Projected Data

Using projected data from PCA, we will now perform clustering again with both k-means and complete linkage.

In [22]:
# re-cluster with kmeans on the projected data
kmeans_a = KMeans(n_clusters=6, random_state=0).fit(projected_a)
fig_a = px.scatter(projected_a, x=0, y=1, color=kmeans_a.labels_, title='KMeans for dataset A with 6 clusters on projected data',
                   color_continuous_scale='solar')
fig_a.show()

kmeans_b = KMeans(n_clusters=6, random_state=0).fit(projected_b)
fig_b = px.scatter(projected_b, x=0, y=1, color=kmeans_b.labels_, title='KMeans for dataset B with 6 clusters on projected data',
                   color_continuous_scale='haline')
fig_b.show()

In [23]:
# re-cluster with agglomerative clustering on the projected data
agg_a = AgglomerativeClustering(n_clusters=6, linkage='complete')
assignments_a = agg_a.fit_predict(projected_a)
fig_a = px.scatter(projected_a,x=0, y=1, color=assignments_a, title='Agglomerative Clustering for dataset A with 6 clusters on projected data',
                   color_continuous_scale='solar')
fig_a.show()

agg_b = AgglomerativeClustering(n_clusters=6, linkage='complete')
assignments_b = agg_b.fit_predict(projected_b)
fig_b = px.scatter(projected_b,x=0, y=1, color=assignments_b, title='Agglomerative Clustering for dataset B with 6 clusters on projected data',
                     color_continuous_scale='haline')
fig_b.show()

# Apply K-Means Clustering to your data set

In [24]:
# Apply PCA on the liver dataset
pca_liver = PCA(data=X_train)
projected_liver = pca_liver.do_pca()
reduced_data = pca_liver.transform(X_train.to_numpy(), n_components=3)
fig = px.scatter_3d(reduced_data, x=reduced_data[:, 0], y=reduced_data[:, 1], z=reduced_data[:, 2],
                    color=y_train.to_numpy().flatten(), title='Liver dataset projected on 3 dimensions',
                    labels={'x': 'X1', 'y': 'X2', 'z': 'X3', 'color': 'Alcoholic Beverage Consumption'})
fig.show()

Mean: [90.26744186 69.93023256 29.79844961 24.59302326 38.50775194]
Standard Deviation: [ 4.52321596 17.15306651 18.47452881 10.01051142 39.73899753]
Eigenvalues: [2.29249695 0.97198351 0.92816507 0.55450711 0.27230261]
Eigenvectors: 
[[-0.23404322 -0.22746188 -0.54525691 -0.58300473 -0.50624624]
 [-0.81250145 -0.41913017  0.3496958   0.20006706 -0.04309662]
 [-0.46141344  0.87656152 -0.03040711 -0.01298116 -0.13283235]
 [ 0.26215636  0.02755323  0.39846119  0.24412776 -0.84388694]
 [ 0.05862461  0.0589213   0.64862303 -0.74853976  0.10985364]]


From the graph above, we can see that our data does not appear to have distinct natural clustering and tends to be noisy. We attempt to do further analysis with k-means clustering and DBSCAN.

In [25]:
k_range = list(range(3, 50))
cluster_quality_metrics = ['elbow', 'mdl', 'ray-turi']
X_train = pca_liver.transform(X_train.to_numpy())

def graph_kmeans(k_range, X, metric: str):
    cluster_qualities = np.zeros(len(k_range))
    for i in k_range:
        # Train a kmeans model on the liver data
        kmeans_liver = KMeans(n_clusters=i, random_state=0)
        kmeans_liver.fit_predict(X)

        # Compute cluster quality
        cq = cluster_quality(X, kmeans_liver, metric=metric)
        cluster_qualities[i-k_range[0]] = cq

    # plot the cluster qualities
    fig_cq = px.line(x=k_range, y=cluster_qualities, title=f'Cluster quality for liver data using {metric}',
                    labels={'x': 'Number of clusters', 'y': 'Cluster quality'})
    fig_cq.show()

for metric in cluster_quality_metrics:
    graph_kmeans(k_range, X_train, metric)
    

## Apply DBSCAN Clustering

We will attempt clustering with another algorithm, DBSCAN, which is density based and does not require us to define the number of clusters.

In [26]:
dbscan = DBSCAN(eps=1, min_samples=2)
clusters = dbscan.fit_predict(reduced_data)
# plot the cluster assignments
fig = px.scatter_3d(reduced_data, x=reduced_data[:, 0], y=reduced_data[:, 1], z=reduced_data[:, 2],
                    color=clusters, color_continuous_scale='Plotly3',
                    title='Clustering of liver data using DBSCAN',)
fig.show()

In [27]:
# evaluate the clustering
print(f'Homogeneity: {homogeneity_score(y_train.to_numpy().flatten(), clusters)}')
print(f'Completeness: {completeness_score(y_train.to_numpy().flatten(), clusters)}')
print(f'Adjusted Rand Index: {adjusted_rand_score(y_train.to_numpy().flatten(), clusters)}')

Homogeneity: 0.09817436598216324
Completeness: 0.15881833206823637
Adjusted Rand Index: 0.15871440279491728


Still, these scores tell us that the clustering is not of high quality.

# Use K-Nearest Neighbor and PCA to classify activity from phone measurements
Finally, using the UCI activity recognition data set, we train two classifiers and compare their performance.

In [28]:
# Load UCI HAR Dataset
X_train = pd.read_csv('project2/data/UCI HAR Dataset/train/X_train.txt', sep='\s+', header=None)
y_train = pd.read_csv('project2/data/UCI HAR Dataset/train/y_train.txt', sep='\s+', header=None)
X_test = pd.read_csv('project2/data/UCI HAR Dataset/test/X_test.txt', sep='\s+', header=None)
y_test = pd.read_csv('project2/data/UCI HAR Dataset/test/y_test.txt', sep='\s+', header=None)

# Merge train and test data and re-split into train and test with 75% train and 25% test
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [29]:
# Build k-Nearst Neighbors classifier on the training data
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train.to_numpy().ravel())
print(f'training score: {knn.score(X_train, y_train)}')
print(f'test score: {knn.score(X_test, y_test)}')


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



training score: 0.9801916105644743
test score: 0.9681553398058252


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



In [30]:
# Do PCA on the training data
pca = PCA(data=X_train)
pca.do_pca()
knn_pca = KNeighborsClassifier(n_neighbors=5)
X_train_projected = pca.transform(X_train, n_components=0.9)
X_test_projected = pca.transform(X_test, n_components=0.9)
print(X_train_projected.shape)
knn_pca.fit(X_train_projected, y_train.to_numpy().ravel())

print(f'training score: {knn_pca.score(X_train_projected, y_train)}')
print(f'test score: {knn_pca.score(X_test_projected, y_test)}')

Mean: [ 0.27485682 -0.01804801 -0.10893692 -0.60471144 -0.50712232 -0.61050949
 -0.63053925 -0.52299668 -0.61232846 -0.46361789 -0.30331473 -0.56078686
  0.52392734  0.38711323  0.59653613 -0.54892222 -0.82359121 -0.90200008
 -0.85400831 -0.68614661 -0.64187695 -0.63790093 -0.09570928 -0.12759217
 -0.15508421 -0.12048605  0.10873482 -0.03555604  0.12276875 -0.0320319
  0.03478455  0.15301409 -0.01749757  0.00396974  0.03815427  0.0343004
 -0.08241367 -0.11913871 -0.1965712   0.09941128  0.66847622  0.00470753
  0.0911893  -0.96532309 -0.95471246 -0.93889215 -0.96612017 -0.95568428
 -0.94021979  0.60822908 -0.00978221  0.09579842  0.68302532  0.01728674
  0.07839253 -0.0992228   0.44477224 -0.71909397 -0.76465177 -0.96831894
 -0.95897598 -0.94486794 -0.67435841 -0.86745338 -0.66697084 -0.5055584
  0.5442787  -0.58230505  0.61968994 -0.34162817  0.32834604 -0.35919674
  0.40765717 -0.43036018  0.45572375 -0.48048611  0.50200256  0.17543436
 -0.11184531  0.08045124  0.07837844  0.00825574

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



training score: 0.9712584153288452
test score: 0.9495145631067962


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



# References

1. Charles Elkan. 2003. Using the triangle inequality to accelerate k-means. In Proceedings of the Twentieth International Conference on International Conference on Machine Learning (ICML'03). AAAI Press, 147–153. https://www.aaai.org/Papers/ICML/2003/ICML03-022.pdf
2. https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/
3. https://en.wikipedia.org/wiki/Chebyshev_distance
4. https://medium.com/swlh/euclidean-distance-matrix-4c3e1378d87f
5. https://www.dabblingbadger.com/blog/2020/2/27/implementing-euclidean-distance-matrix-calculations-from-scratch-in-python
6. https://medium.com/@muhammadammarabid01/k-nearest-neigbors-knn-basic-algorithm-from-scratch-in-python-8471b655a013
7. https://machinelearningmastery.com/distance-measures-for-machine-learning/
8. https://www.fmrib.ox.ac.uk/datasets/techrep/tr00mj2/tr00mj2/node24.html#:~:text=For%20the%20method%20proposed%20here,%3D%20n%20(IQR)%2F100.
9. http://www.jtrive.com/determining-histogram-bin-width-using-the-freedman-diaconis-rule.html
10. Ray, Siddheswar & Turi, Rose. (2000). Determination of Number of Clusters in K-Means Clustering and Application in Colour Image Segmentation. Proceedings of the 4th International Conference on Advances in Pattern Recognition and Digital Techniques (ICAPRDT'99). 1.
11. https://www.statology.org/confusion-matrix-python/
12. https://towardsdatascience.com/cosine-similarity-how-does-it-measure-the-similarity-maths-behind-and-usage-in-python-50ad30aad7db
13. https://en.wikipedia.org/wiki/Minkowski_distance

## Data Source
1. Liver Disorders. (1990). UCI Machine Learning Repository. https://archive-beta.ics.uci.edu/dataset/60/liver+disorders
2. https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones